# Baseline

**Metodologia**: Como baseline, iremos recomendar livros aleatórios.

## 0. Setup

In [1]:
%load_ext autotime

time: 164 µs (started: 2024-02-18 23:38:16 -03:00)


In [2]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

time: 479 ms (started: 2024-02-18 23:38:17 -03:00)


In [3]:
#---- Data path:

data_path = '../../03-data/'

#---- Template dos gráficos:

pio.templates.default = 'plotly_white'

time: 147 ms (started: 2024-02-18 23:38:17 -03:00)


## 1. Extração dos dados

### 1.1. Dados de treino e teste

In [4]:
df_train = pd.read_parquet(data_path + '02-processed/01-train_data.parquet')
df_test = pd.read_parquet(data_path + '02-processed/02-test_data.parquet')

time: 343 ms (started: 2024-02-18 23:38:17 -03:00)


### 1.2. Full data

In [5]:
df_full = pd.read_parquet(data_path + '02-processed/03-full_data.parquet')

time: 982 ms (started: 2024-02-18 23:38:17 -03:00)


## 2. Baseline

### 2.1. Separando os livros em uma lista para a escolha aleatória

In [6]:
list_of_books = df_full\
    .groupby('Book-Title', as_index = False)\
    .agg(
        media_rating = ('Book-Rating', 'mean'),
        frequencia = ('ISBN', 'count')
    )\
    .query('frequencia > 10')['Book-Title']\
    .to_list()

time: 940 ms (started: 2024-02-18 23:38:18 -03:00)


In [7]:
from random import sample

random_book = sample(list_of_books, 5)
random_book

['The Eleventh Plague',
 'Retribution',
 'The Scottish Bride (Bride Trilogy (Paperback))',
 'The Picture of Dorian Gray and Other Short Stories',
 'Centaur Aisle']

time: 4.37 ms (started: 2024-02-18 23:32:36 -03:00)


In [8]:
def choose_random_books(n: int = 5):

    random_book = sample(list_of_books, n)

    return random_book

time: 822 µs (started: 2024-02-18 23:32:36 -03:00)


In [9]:
df_test['recommendations'] = df_test.apply(lambda x: choose_random_books(n = 20), axis = 1)

time: 438 ms (started: 2024-02-18 23:31:35 -03:00)


In [10]:
df_test['flag_recommendations'] = df_test.apply(lambda x: x['Book-Title'] in x['recommendations'], axis = 1)

time: 222 ms (started: 2024-02-18 23:31:38 -03:00)


In [12]:
precision_k = df_test.query('flag_recommendations == 1').shape[0] / df_test.shape[0] * 100

precision_k

0.03544842254519674

time: 11.4 ms (started: 2024-02-18 23:32:00 -03:00)


In [7]:
from random import sample

def choose_random_books(n: int = 5):

    random_book = sample(list_of_books, n)

    return random_book

time: 662 µs (started: 2024-02-18 23:38:25 -03:00)


In [8]:
def calculate_precision_k_baseline(df: pd.DataFrame, col_books: str, lst_recommendations: list, k: int = 10):

    df['recommendations'] = df.apply(lambda x: choose_random_books(n = k), axis = 1)

    df['flag_recommendations'] = df.apply(lambda x: x[col_books] in x['recommendations'], axis = 1)

    precision_k = df.query('flag_recommendations == 1').shape[0] / df.shape[0] * 100

    return df, precision_k

time: 1.27 ms (started: 2024-02-18 23:38:25 -03:00)


In [9]:
df, precision_k = calculate_precision_k_baseline(df = df_test, col_books = 'Book-Title', lst_recommendations = list_of_books, k = 20)

time: 663 ms (started: 2024-02-18 23:38:27 -03:00)


In [10]:
df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,recommendations,flag_recommendations
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5,"[The Lasko Tangent, The Ear, the Eye, and the ...",False
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4,"[New York Dead, Therapy, Thank You for Smoking...",False
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3,"[My Sister's Keeper : A Novel (Picoult, Jodi),...",False
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4,"[Family: The Ties That Bind and Gag, Dreamer, ...",False
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3,"[The Green Knight, My Name Is Asher Lev, The M...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22563,276680,1931561648,9,The Time Traveler's Wife (Today Show Book Club...,Audrey Niffenegger,2003,MacAdam/Cage Publishing,55.0,hopewell junction,new york,usa,91,"[Fruitcake (Mira), House of Smoke, The Snowman...",False
22564,276681,1931333246,10,Chorus,Kurt Messick,2003,Dry Bones Press,43.0,chicago,illinois,usa,11,"[Winter Moon, The Day Trader, Because It Is Bi...",False
22565,276683,0553278231,9,Cannery Row,John Steinbeck,1986,Bantam Books,NaN,charlotte,north carolina,usa,3,"[Slightly Married (Get Connected Romances), 21...",False
22566,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,NaN,fort lee,new jersey,usa,29,"[Picture Postcard, Of Mice and Men (Penguin Gr...",False


time: 31.9 ms (started: 2024-02-18 23:38:27 -03:00)


In [11]:
precision_k

0.044310528181495924

time: 2.21 ms (started: 2024-02-18 23:38:28 -03:00)


# TO-DO

- Entender o recall@k
- Calcular ao recall@k
- MAR@K
- MAP@K
- https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
- https://cran.r-project.org/web/packages/recometrics/recometrics.pdf
- https://medium.com/@judaikawa/building-and-evaluating-a-recommender-system-for-implicit-feedback-59495d2077d4